In [8]:
import pandas as pd

def process_age_groups(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 연령대 그룹 매핑 딕셔너리 생성
    age_mapping = {
        '19_24': '20대',
        '25_29': '20대',
        '30_34': '30대',
        '35_39': '30대',
        '40_44': '40대',
        '45_49': '40대',
        '50_54': '50대',
        '55_59': '50대',
        '60_80': '60대 이상'
    }
    
    # 원하는 연령대만 필터링
    df = df[df['age'].isin(age_mapping.keys())]
    
    # 새로운 연령대 컬럼 생성
    df['age_group'] = df['age'].map(age_mapping)
    
    # 일자별, 새로운 연령대별로 score 합산
    result = df.groupby(['date', 'name', 'age_group'])['score'].sum().reset_index()
    
    # 피벗 테이블로 변환하여 보기 좋게 정리
    pivot_result = result.pivot(index=['date', 'name'], 
                              columns='age_group', 
                              values='score').reset_index()
    
    # 컬럼 순서 정리
    column_order = [ 'date', 'name', '20대', '30대', '40대', '50대', '60대 이상']
    pivot_result = pivot_result[column_order]
    
    return pivot_result



In [9]:
# 광어
result_df = process_age_groups('../../data/raw/nst_광어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_광어_trend_2025-01-17.csv', index=False)

In [10]:
# 농어 
result_df = process_age_groups('../../data/raw/nst_농어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_농어_trend_2025-01-17.csv', index=False)

In [11]:
# 대게 
result_df = process_age_groups('../../data/raw/nst_방어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_방어_trend_2025-01-17.csv', index=False)

In [12]:
# 방어 
result_df = process_age_groups('../../data/raw/nst_방어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_방어_trend_2025-01-17.csv', index=False)

In [13]:
# 연어
result_df = process_age_groups('../../data/raw/nst_연어_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_연어_trend_2025-01-17.csv', index=False)

In [14]:
# 우럭 
result_df = process_age_groups('../../data/raw/nst_우럭_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_우럭_trend_2025-01-17.csv', index=False)

In [15]:
# 참돔 
result_df = process_age_groups('../../data/raw/nst_참돔_trend_2025-01-17.csv')
result_df.to_csv('그룹화_nst_참돔_trend_2025-01-17.csv', index=False)

In [4]:
import pandas as pd

def calculate_item_daily_average(file_path):
    # CSV 파일 읽기
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 날짜와 어종별로 그룹화하여 시장 평균 계산
    daily_avg = df.groupby(['priceDate', 'item'])['avgPrice'].mean().reset_index()
    
    # 결측치를 앞뒤 값으로 채우기
    daily_avg = daily_avg.sort_values(['item', 'priceDate'])
    daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
    daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='bfill')
    
    return daily_avg

# 파일 처리
file_path = '../../data/processed/item_price_lag_filled.csv'
result_df = calculate_item_daily_average(file_path)

# 결과 저장
output_file = 'daily_item_average_prices.csv'
result_df.to_csv(output_file, index=False)
print(f"처리된 데이터가 {output_file}에 저장되었습니다.")

# 결과 미리보기
print("\n처리된 데이터 미리보기:")
print(result_df.head())

처리된 데이터가 daily_item_average_prices.csv에 저장되었습니다.

처리된 데이터 미리보기:
     priceDate item  avgPrice
19  2015-02-25   광어   25000.0
23  2015-02-26   광어   25000.0
27  2015-02-27   광어   25000.0
31  2015-02-28   광어   25000.0
35  2015-03-01   광어   25000.0


C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:12: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:12: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily_avg['avgPrice'] = daily_avg.groupby('item')['avgPrice'].fillna(method='ffill')
C:\Users\user\AppData\Local\Temp\ipykernel_4516\2890981007.py:13: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  daily_avg['avgPrice'] = daily_avg

In [5]:
import pandas as pd
import DateTime

def transform_market_data(file_path):
    # 데이터 읽기
    df = pd.read_csv(file_path)
    
    # 시장별 더미변수 생성 (0, 1로 인코딩)
    market_dummies = pd.get_dummies(df['market'], prefix='m').astype(int)
    
    # 원본 데이터와 더미변수 결합
    result = pd.concat([
        df[['priceDate', 'item']], 
        market_dummies,
        df[['avgPrice', 'avgPrice_lag_1']]
    ], axis=1)
    
    # 날짜와 어종으로 정렬
    result = result.sort_values(['priceDate', 'item'])
    
    # 변환된 데이터 저장
    output_file = 'transformed_market_data.csv'
    result.to_csv(output_file, index=False)
    print(f"생성된 파일: {output_file}")
    print("\n처음 10개 컬럼:", result.columns[:10].tolist())
    
    return result



In [6]:
# Execute
df = transform_market_data('../../data/processed/item_price_lag_filled.csv')

생성된 파일: transformed_market_data.csv

처음 10개 컬럼: ['priceDate', 'item', 'm_가락시장', 'm_강서농수산물시장', 'm_구리농수산물시장', 'm_노량진 1층', 'm_노량진 2층', 'm_마포농수산물시장', 'm_부산민락어민활어직판장', 'm_소래포구종합어시장']


In [9]:
import pandas as pd

def split_market_data():
    df = pd.read_csv('item_price_oneHot.csv')
    
    for fish in df['item'].unique():
        fish_df = df[df['item'] == fish]
        output_file = f'{fish}_price_oneHot.csv'
      # fish_df = fish_df.drop('item', axis=1)
        fish_df.to_csv(output_file, index=False)
        print(f'{fish} 데이터 생성 완료: {len(fish_df)}개 행')



In [10]:
split_market_data()

대게 데이터 생성 완료: 27677개 행
광어 데이터 생성 완료: 31515개 행
농어 데이터 생성 완료: 30651개 행
연어 데이터 생성 완료: 29588개 행
참돔 데이터 생성 완료: 27149개 행
방어 데이터 생성 완료: 11205개 행
우럭 데이터 생성 완료: 6081개 행


In [17]:
import pandas as pd

def merge_features_price():
    # 두 데이터셋 로드
    features_df = pd.read_csv('../../data/features/참돔_timelagged_features.csv')
    price_df = pd.read_csv('../../data/features/oneHot/참돔_price_oneHot.csv')
    
    # 날짜 컬럼명 통일
    price_df = price_df.rename(columns={'dmdpriceDate': 'date'})
    
    # 날짜 기준으로 데이터 병합
    merged_df = pd.merge(price_df, features_df, on='date', how='left')
    
    # CSV 파일로 저장
    output_file = '참돔_price_features.csv'
    merged_df.to_csv(output_file, index=False)
    print(f'생성된 파일: {output_file}')
    print(f'전체 컬럼 수: {len(merged_df.columns)}')

merge_features_price()

생성된 파일: 참돔_price_features.csv
전체 컬럼 수: 35


In [20]:
import pandas as pd

def merge_noryangjin_data():
    # 데이터 로드
    df = pd.read_csv('광어_price_features.csv')
    
    # 노량진 1층과 2층 데이터 추출
    floor1 = df[df['m_노량진 1층'] == 1]
    floor2 = df[df['m_노량진 2층'] == 1]
    
    # 필요한 컬럼만 선택하여 데이터프레임 생성
    floor1_prices = floor1[['date', 'avgPrice', 'avgPrice_lag_1']]
    floor2_prices = floor2[['date', 'avgPrice', 'avgPrice_lag_1']]
    
    # 날짜별로 평균 계산
    merged_prices = pd.merge(floor1_prices, floor2_prices, on='date', how='outer', suffixes=('_1층', '_2층'))
    merged_prices['avgPrice'] = merged_prices[['avgPrice_1층', 'avgPrice_2층']].mean(axis=1)
    merged_prices['avgPrice_lag_1'] = merged_prices[['avgPrice_lag_1_1층', 'avgPrice_lag_1_2층']].mean(axis=1)
    
    # 원본 데이터에서 노량진 1층, 2층 데이터 제거
    df_filtered = df[~((df['m_노량진 1층'] == 1) | (df['m_노량진 2층'] == 1))].copy()
    
    # 새로운 노량진 데이터 생성
    noryangjin_data = df_filtered.iloc[0:len(merged_prices)].copy()
    noryangjin_data.loc[:, 'date'] = merged_prices['date'].values
    noryangjin_data.loc[:, 'avgPrice'] = merged_prices['avgPrice'].values
    noryangjin_data.loc[:, 'avgPrice_lag_1'] = merged_prices['avgPrice_lag_1'].values
    
    # 모든 시장 칼럼을 0으로 설정
    market_cols = [col for col in df.columns if col.startswith('m_')]
    for col in market_cols:
        noryangjin_data[col] = 0
    
    # 노량진 칼럼 추가
    noryangjin_data['m_노량진'] = 1
    
    # 기존 노량진 1층, 2층 칼럼 삭제
    df_filtered = df_filtered.drop(['m_노량진 1층', 'm_노량진 2층'], axis=1)
    noryangjin_data = noryangjin_data.drop(['m_노량진 1층', 'm_노량진 2층'], axis=1)
    
    # 데이터 합치기
    final_df = pd.concat([df_filtered, noryangjin_data], axis=0)
    
    # 날짜순으로 정렬
    final_df = final_df.sort_values('date')
    
    # m_노량진 칼럼 위치 조정
    cols = final_df.columns.tolist()
    cols.remove('m_노량진')
    cols.insert(3, 'm_노량진')
    final_df = final_df[cols]
    
    # 저장
    final_df.to_csv('광어_price_features_noryangjin_merged.csv', index=False)
    print('노량진 데이터 통합 및 칼럼 재정렬 완료')
    

In [21]:
merge_noryangjin_data()

노량진 데이터 통합 및 칼럼 재정렬 완료
